In [1]:
!pip install selenium

In [2]:
import selenium

In [3]:
from selenium import  webdriver

In [4]:
driver_path = "C:/Users/NKA/Downloads/chromedriver.exe"

In [147]:
# Créer un navigateur
# téléchargerr le driver
# * Firefox : https://github.com/mozilla/geckodriver/releases
# Chrome : https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(executable_path= driver_path)

In [148]:
driver.get("https://www.google.com/flights?hl=fr&gl=fr")

In [149]:
# page = driver.page_source
# page[:1000]

In [150]:
origin_airport = driver.find_element_by_css_selector(
    "div[data-flt-ve='origin_airport'][class*='flt-input']")

In [151]:
# len(origin_airport)

In [152]:
origin_airport.get_attribute("outerHTML")

'<div data-flt-ve="origin_airport" jsaction="jsl._;f_f:jsl._;ica_bc:jsl._;keydown:jsl._" class="flt-input gws-flights-form__input-container gws-flights__flex-box gws-flights-form__airport-input gws-flights-form__empty gws-flights-form__swapper-right" role="presentation" jsan="0.data-flt-ve,22.jsaction,t-ch-jmKqQoDY,7.flt-input,7.gws-flights-form__input-container,7.gws-flights__flex-box,7.gws-flights-form__airport-input,7.gws-flights-form__empty,7.gws-flights-form__swapper-right,0.role,21.jssc" jstcache="573"><div jscontroller="pSNSIe" class="gws-flights-widgets-inputcapturearea__capture-area-wrapper" jsaction="rcuQ6b:npT2md" jstcache="468"><span jsname="bOjMyf" style="display:none" aria-live="polite" jsvs="\'bOjMyf\';" jsan="0.jsname,5.display,0.aria-live,t-uU8ghUBFeaU" id="flt-ica-1" jstcache="492"> <span jsan="t-qoTycjZOEvo,21.jssc" jstcache="489">D\'où partez-vous&nbsp;?</span> <jsl jstcache="490"></jsl> </span><textarea jsname="LlUqIb" class="gws-flights-widgets-inputcapturearea__c

In [153]:
origin_airport.click() # Pour cliquer et faire apparaître le formulaire 

In [154]:
# origin_airport.send_keys("abidjan")

In [155]:
origin_airport2 = driver.find_element_by_css_selector(
    "input[placeholder*='partez-vous']")

In [156]:
origin_airport2.send_keys("abidjan")

In [157]:
origin_airport3 = driver.find_element_by_css_selector(
    ".fsapp-option-content .fsapp-option-city-name")

In [158]:
origin_airport3.click()

In [159]:
destination_airport = driver.find_element_by_css_selector(
    "div[data-flt-ve='destination_airport'][class*='flt-input']")

In [160]:
destination_airport.click()

In [161]:
destination_airport2 = driver.find_element_by_css_selector(
    "input[placeholder*='allez-vous']")

In [162]:
destination_airport2.send_keys("dakar")

In [163]:
destination_airport3 = driver.find_element_by_css_selector(
    ".fsapp-option-content .fsapp-option-city-name")

In [164]:
destination_airport3.click()

In [165]:
search_button = driver.find_element_by_css_selector(
                        "floating-action-button.gws-flights-form__search-button")

In [166]:
search_button.click()

In [167]:
expand_button = driver.find_element_by_css_selector(
                        ".gws-flights-results__dominated-toggle")

In [168]:
expand_button.click()